In [1]:
# Mount "My Drive" into /content/drive
from google.colab import drive

google_drive_dir = "Final_project"  # @param
#bml-notebooks/
drive.mount('/content/drive')

mount_point = "/content/drive/My Drive/{}".format(google_drive_dir)

# Change the root directory to your mount_point
% cd '$mount_point'

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1uHceact8nZzLehIO7J8WmUhJnJWKOf8w/Final_project


# <span style="color: royalblue;">Load Data</span>
Load the image data with labels.

In [2]:
import numpy as np
from src.loadData import loadFer2013
from keras.utils import to_categorical


In [3]:
X_train,X_val,X_test,y_train,y_val,y_test = loadFer2013('fer2013/icml_face_data.csv')

y_train_onehot = to_categorical(y_train)
y_val_onehot = to_categorical(y_val)
y_test_onehot = to_categorical(y_test)

/content/drive/.shortcut-targets-by-id/1uHceact8nZzLehIO7J8WmUhJnJWKOf8w/Final_project/src/loadData.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['emotion'][data['emotion']>1]-=1


In [4]:
y_train_onehot.shape, y_val_onehot.shape, y_test_onehot.shape

((28709, 6), (3589, 6), (3589, 6))

In [5]:
X_train.shape, y_train.shape

((28709, 48, 48), (28709,))

In [6]:
emotions = {0: 'Angry', 1:'Fear', 2: 'Happy', 3: 'Sad', 4: 'Surprise', 5: 'Neutral'}

In [7]:
class_names = list(emotions.values())
class_names

['Angry', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Extract features

In [11]:
X_val.shape

(3589, 48, 48)

In [12]:
X_data = np.vstack((X_train,X_val))
X_data.shape

(32298, 48, 48)

In [13]:
X_data = X_data.reshape(32298,-1)
X_data.shape

(32298, 2304)

In [14]:
X_test.shape

(3589, 48, 48)

In [15]:
h,w = X_train[0].shape
h,w

(48, 48)

In [16]:
import time
from sklearn.decomposition import PCA

n_components = 150

print("Extracting the top %d eigenfaces from %d faces"
      % (n_components, X_data.shape[0]))
t0 = time.time()
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(X_data)
print("done in %0.3fs" % (time.time() - t0))

eigenfaces = pca.components_.reshape((n_components, h, w))

print("Projecting the input data on the eigenfaces orthonormal basis")
t0 = time.time()
X_train_pca = pca.transform(X_data)
X_test_pca = pca.transform(X_test.reshape(3589,-1))
print("done in %0.3fs" % (time.time() - t0))

Extracting the top 150 eigenfaces from 32298 faces
done in 15.784s
Projecting the input data on the eigenfaces orthonormal basis
done in 1.088s


In [17]:
X_train_pca.shape

(32298, 150)

# SVM Classification

In [19]:

y_label = np.vstack((y_train_onehot,y_val_onehot))
y_label.shape

(32298, 6)

In [ ]:
from sklearn.svm import SVC
svm=SVC(kernel='rbf', class_weight='balanced',C=1)
st = time.time()
svm.fit(X_train_pca, y_label)
print(time.time()-st)
svm.score(X_train_pca, y_label)

#Evaluation

In [23]:
import pickle
svm = pickle.load(open('saved_model/eignface_svmC1.pkl', 'rb'))

In [24]:
# Quantitative evaluation of the model quality on the test set
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print("Predicting people's names on the test set")
t0 = time.time()
y_pred = svm.predict(X_test_pca)
print("done in %0.3fs" % (time.time() - t0))

print(classification_report(y_test, y_pred, target_names=class_names))
print(confusion_matrix(y_test, y_pred, labels=range(len(class_names))))

Predicting people's names on the test set
done in 32.136s
              precision    recall  f1-score   support

       Angry       0.29      0.29      0.29       467
        Fear       0.40      0.34      0.37       552
       Happy       0.60      0.60      0.60       895
         Sad       0.37      0.39      0.38       653
    Surprise       0.65      0.62      0.63       415
     Neutral       0.40      0.45      0.42       607

    accuracy                           0.46      3589
   macro avg       0.45      0.45      0.45      3589
weighted avg       0.46      0.46      0.46      3589

[[137  58  94  91  22  65]
 [ 75 189  68  98  44  78]
 [ 74  50 541 104  26 100]
 [ 95  76  78 253  24 127]
 [ 23  44  28  31 257  32]
 [ 68  54  88 102  24 271]]


In [25]:
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred))
print(metrics.f1_score(y_test, y_pred,average='micro'))


0.4591808303148509
0.4591808303148509
